In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

# SCOPES: Gmail read-only
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
token_path = os.getenv("token_path")
credentials_path = os.getenv("credentials_path")

In [ ]:
def get_message_metadata(service, user_id='me', max_results=5):
    results = service.users().messages().list(userId=user_id, maxResults=max_results, q="after:2025/08/08").execute()
    messages = results.get('messages', [])

    for _, msg in enumerate(messages):
        msg_data = service.users().messages().get(userId=user_id, id=msg['id'], format='metadata', metadataHeaders=['Subject', 'From', 'Date']).execute()
        headers = msg_data.get('payload', {}).get('headers', [])

        email_info = {header['name']: header['value'] for header in headers if header['name'] in ['Subject', 'From', 'Date']}
        print(f"From: {email_info.get('From')}")
        print(f"Subject: {email_info.get('Subject')}")
        print(f"Date: {email_info.get('Date')}")

In [2]:
import gzip, msgspec
import pandas as pd
path = "C:/airflow-docker/data/15-08-2025-10-41.json.gz"
#Decompress and load
with gzip.open(path, 'rb') as f:
    decompressed_bytes = f.read()
decompressed_data = msgspec.msgpack.decode(decompressed_bytes)

df =pd.DataFrame(decompressed_data)
payload = df.loc[3,["Payload"]].values
df.head(2)

,Id,Payload
0,1988691375a9beb7,"{'partId': '', 'mimeType': 'multipart/alternat..."
1,19885fd9b18c5065,"{'partId': '', 'mimeType': 'multipart/mixed', ..."


In [3]:
# x = payload[0].get("headers", [])
# for h in x:
#     print(h)

In [4]:
from dags.utils import extract_headers, decode_body 
for payload in decompressed_data["Payload"][:1]:
    subject, sender = extract_headers(payload)
    body_text = decode_body(payload, prefer_plain=True).strip()

    print(f"\n--- Email ---")
    print(f"From: {sender}")
    print(f"Subject: {subject}")
    print(f"Body:\n{body_text}")


--- Email ---
From: Braintrust
Subject: Enhance Your Career with Braintrust's Career Help
Body:
Satwik,

Ready to take your career to the next level? Dive into Career Help https://usebraintrust.acemlnb.com/lt.php?x=3TZy~GDJKaSc6p39y_A6VhOh1XIiutfzjewzk8c7VXCZ6576-ky.0uFr1nRziNfuk_YwXXIWI3Og65Xt-x6.hFhv2nEik_f0lb, where you can ask questions, share insights, and network with industry leaders. Whether you're seeking advice on transitioning to a new field, finding the best tutorials, or improving your skills, our community supports you.

Ask a question https://usebraintrust.acemlnb.com/lt.php?x=3TZy~GDJKaSc6p39y_A6VhOh1XIiutfzjewzk8c7VXCZ6576-ky.0uFr1nRziNfuk_YwXXIWI3Og65Xt-x6.hFhv2nEik_f0lb 

In Career Help, you can:

* Ask questions: Get answers from experienced professionals.

* Find mentorship: Give or get mentorship and personalized advice.

* Network with peers: Connect with peers and industry leaders.

* Join curated Spaces: Participate in curated groups focusing on your interests

In [5]:
from dags.utils import preprocess_email_body
# Example usage
modified_text = preprocess_email_body(body_text)
print(len(body_text), len(modified_text))

2214 1147
